In [20]:
from utils import flat_subreddits

PARENT_PATH = os.path.abspath(os.path.dirname(__file__))

import requests
import libtorrent as lt
import time
import zstandard as zstd
from pathlib import Path
import os

In [37]:
torrent_link = "https://academictorrents.com/download/1614740ac8c94505e4ecb9d88be8bed7b6afddd4.torrent"
torrent_file = f"{PARENT_PATH}/data/downloads/downloaded_content.torrent"

if not os.path.exists(torrent_file):
    with requests.get(torrent_link, stream=True) as r:
        r.raise_for_status()
        with open(torrent_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print("Torrent file downloaded.")
else:
    print("Torrent file already exists, skipping download.")

Torrent file downloaded.


In [41]:
session = lt.session()
info = lt.torrent_info(f"{PARENT_PATH}/data/downloads/downloaded_content.torrent")
matching_files = []
file_priorities = {}
for index, file in enumerate(info.files()):
    file_name = os.path.basename(file.path)
    base_name = file_name.replace('_submissions.zst', '').replace('_comments.zst', '')
    file_priorities[index] = 0

    if base_name in flat_subreddits:
        print(f"Found matching file: {base_name}")
        matching_files.append(file_name)
        file_priorities[index] = 1
        
if not matching_files:
    print("No files in the torrent match the provided reddits list.")
else:
    # Check if files already exist
    all_files_exist = True
    for file_name in matching_files:
        expected_path = Path(f"{PARENT_PATH}/data/downloads/reddit/subreddits23/{file_name}")
        if not expected_path.exists():
            all_files_exist = False
            break
    
    if all_files_exist:
        print("All matching files already exist. Skipping download.")
    else:
        print("Some files missing. Starting download...")
        handle = session.add_torrent({'ti': info, 'save_path': f"{PARENT_PATH}/data/downloads"})
        for index in file_priorities:
            handle.file_priority(index, file_priorities[index])
        while not handle.is_seed():
            s = handle.status()
            progress = s.progress * 100
            print(f"Download progress: {progress:.2f}%")
            if progress >= 100:
                break
            time.sleep(1)
        print("Download complete for matching files.")
print("Script finished.")

C:\Users\andrew\AppData\Local\Temp\ipykernel_4112\2015518000.py:5: DeprecationWarning: __iter__ is deprecated
  for index, file in enumerate(info.files()):


Found matching file: AskALiberal
Found matching file: AskALiberal
Found matching file: AskConservatives
Found matching file: AskConservatives
Found matching file: Conservative
Found matching file: Conservative
Found matching file: PoliticalDiscussion
Found matching file: PoliticalDiscussion
Found matching file: SocialDemocracy
Found matching file: SocialDemocracy
Found matching file: centrist
Found matching file: centrist
Some files missing. Starting download...


C:\Users\andrew\AppData\Local\Temp\ipykernel_4112\2015518000.py:33: DeprecationWarning: is_seed() is deprecated
  while not handle.is_seed():


Download progress: 0.01%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%
Download progress: 0.00%


KeyboardInterrupt: 

In [43]:
output_dir = Path(f"{PARENT_PATH}/data/extracted")
if not output_dir.exists():
    output_dir.mkdir(parents=True)

download_dir = Path(f"{PARENT_PATH}/data/downloads/reddit/subreddits24")
zst_files = list(download_dir.glob('*.zst'))

print(f"Found {len(zst_files)} .zst files to extract")

for zst_file in zst_files:

    output_file = output_dir / zst_file.stem
    
    if not output_file.exists():
        print(f"Extracting {zst_file.name}...")
        
        with open(zst_file, 'rb') as compressed:
            dctx = zstd.ZstdDecompressor()
            with open(output_file, 'wb') as destination:
                dctx.copy_stream(compressed, destination)
        
        print(f"Extracted to {output_file}")
    else:
        print(f"Skipping {zst_file.name} - already extracted")

print("All files processed")

Found 2 .zst files to extract
Extracting AskConservatives_comments.zst...
Extracted to C:\Users\andrew\Desktop\sicss_project\data\extracted\AskConservatives_comments
Extracting AskConservatives_submissions.zst...
Extracted to C:\Users\andrew\Desktop\sicss_project\data\extracted\AskConservatives_submissions
All files processed
